In [ ]:
! pip install chromadb
! pip install langchain
! pip install pdfplumber
! pip install  PyPDF4
! pip install huggingface_hub


In [2]:
from langchain.vectorstores import Chroma

In [20]:
import pdfplumber
import PyPDF4
import re
import os
import sys
from typing import Callable, List, Tuple, Dict

from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

In [21]:
from dotenv import load_dotenv
from langchain.embeddings import HuggingFaceHubEmbeddings

In [59]:
!python pdf_processor.py

You're using a different task than the one specified in the repository. Be sure to know what you're doing :)


In [36]:
extract_metadata_from_pdf("/content/fundamental_rights_upsc.pdf")

{'title': 'Download Fundamental Rights UPSC Notes PDF',
 'author': "BYJU'S EXAM PREP",
 'creation_date': "D:20220601123702+05'30'"}

In [ ]:
extract_pages_from_pdf("/content/fundamental_rights_upsc.pdf")

In [36]:
pages, metadata = parse_pdf("/content/fundamental_rights_upsc.pdf")

# Print the extracted metadata
print(metadata)


{'title': 'Download Fundamental Rights UPSC Notes PDF', 'author': "BYJU'S EXAM PREP", 'creation_date': "D:20220601123702+05'30'"}


In [40]:
raw_pages, metadata = parse_pdf("/content/fundamental_rights_upsc.pdf")

In [41]:
cleaning_functions = [
        merge_hyphenated_words,
        fix_newlines,
        remove_multiple_newlines,
    ]

In [42]:
cleaned_text_pdf = clean_text(raw_pages, cleaning_functions)


In [39]:
merge_hyphenated_words("hi-\nhello")

'hihello'

In [40]:
fix_newlines("this is new line.\nthis is second")

'this is new line. this is second'

In [ ]:
clean_text(pages,[merge_hyphenated_words,fix_newlines,remove_multiple_newlines])

In [22]:

def text_to_docs(text: List[str], metadata: Dict[str, str]) -> List[Document]:
    """Converts list of strings to a list of Documents with metadata."""
    doc_chunks = []

    for page_num, page in text:
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""],
            chunk_overlap=200,
        )
        chunks = text_splitter.split_text(page)
        for i, chunk in enumerate(chunks):
            doc = Document(
                page_content=chunk,
                metadata={
                    "page_number": page_num,
                    "chunk": i,
                    "source": f"p{page_num}-{i}",
                    **metadata,
                },
            )
            doc_chunks.append(doc)

    return doc_chunks

In [1]:
import sqlite3

# Connect to the database
conn = sqlite3.connect("/content/src/data/chroma/chroma.sqlite3")

# Create a cursor
cursor = conn.cursor()

# Execute a query to get table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch the table names
table_names = cursor.fetchall()

# Print the fetched table names
for table in table_names:
    print(table[0])


NameError: ignored

In [60]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='upsc-polity';")
table_exists = cursor.fetchone() is not None
table_exists

False

In [ ]:
import sqlite3

# Connect to the database
conn = sqlite3.connect("/content/src/data/chroma/chroma.sqlite3")

# Create a cursor
cursor = conn.cursor()

# Query the database
cursor.execute("SELECT * FROM embeddings")

# Fetch the results
results = cursor.fetchall()

# Print the results
for result in results:
    print(result[0], result[1])

In [ ]:
import sqlite3

# Connect to the database
conn = sqlite3.connect("/content/src/data/chroma/chroma.sqlite3")

# Create a cursor
cursor = conn.cursor()

# Query the database
cursor.execute("PRAGMA table_info(embeddings)")

# Fetch the results
results = cursor.fetchall()

# Print the results
for result in results:
    print(result[1])


#UNIT TEST FOR EMBEDDING VECTORS

In [ ]:
import sqlite3
import numpy as np

# Connect to the database
conn = sqlite3.connect("/content/src/data/chroma/chroma.sqlite3")

# Create a cursor
cursor = conn.cursor()

# Query the database
cursor.execute("SELECT embedding_id FROM embeddings")

# Fetch the results
results = cursor.fetchall()

# Print the results
for result in results:
    embedding_id = result[0].encode("utf-8")
    embedding_vector = np.frombuffer(embedding_id, dtype=np.float32)
    print(embedding_vector)


In [70]:
# Print the first embedding vector
first_embedding_vector = results[0][0].encode("utf-8")
first_embedding = np.frombuffer(first_embedding_vector, dtype=np.float32)
print(first_embedding)
print(len(embedding_vector))

[5.2896574e-08 5.4642762e-05 4.2188891e-08 2.5782576e-09 7.9964824e+20
 1.2915932e-11 1.0488790e-08 3.2722347e+21 3.2500992e+21]
9


# UNIT TEST FOR VECTOR STORE DB

In [22]:
!python pdf_processor.py

In [8]:
from pdf_processor import *
from dotenv import load_dotenv
load_dotenv()

True

In [23]:
pdf_file_path = os.getenv("INPUTPDF_PATH")
db = generate_embeds(pdf_file_path)

You're using a different task than the one specified in the repository. Be sure to know what you're doing :)


#unit test for QUERY

In [ ]:
query = "Article 22"
matching_docs = db.similarity_search(query)

matching_docs[0]
